In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df_trn = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv")
df_tst = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_tst.head

In [ ]:
df_trn = pd.get_dummies(df_trn, columns=["store", "country","product"])
df_tst = pd.get_dummies(df_tst, columns=["store", "country","product"])

In [ ]:
df_tst.shape

In [ ]:
df_trn.columns, df_tst.columns, df_trn.shape, df_tst.shape

In [ ]:
from dateutil.easter import easter
easter_l = []
easter_year=[]
for iy in (2015,2016,2017,2018,2019):
     easter_l.append( (pd.to_datetime(easter(iy)).day_of_year))
     easter_year.append(iy)
        
easter_dict = {"year":easter_year, "easter_day":easter_l}
easter_df = pd.DataFrame.from_dict(easter_dict)
easter_df.set_index("year",inplace=True)
easter_df

In [ ]:
def gettimeFeatures(df):
    # easter day 
    pd.to_datetime(easter(2015)).day_of_year
    
    df["day_of_year"]=pd.to_datetime(df.date).dt.day_of_year
    df["day_of_month"] = pd.to_datetime(df.date).dt.day
    df["week"]=pd.to_datetime(df.date).dt.isocalendar().week
    df["quarter"]=pd.to_datetime(df.date).dt.quarter
    df["month"]=pd.to_datetime(df.date).dt.month
    df["year"]=pd.to_datetime(df.date).dt.year
    df["weekd"]=pd.to_datetime(df.date).dt.weekday
    df["weekend"]=(pd.to_datetime(df.date).dt.weekday>4).astype(int) ## weekday range from 0 to 6. 
    
    ## fourier features 
    import math
    dayofyear=df_trn.day_of_year

    for k in range(1, 32, 4):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'Finland_sin{k}'] = df[f'sin{k}'] * df['country_Finland']
        df[f'Finland_cos{k}'] = df[f'cos{k}'] * df['country_Finland']
        df[f'Norway_sin{k}'] = df[f'sin{k}'] * df['country_Norway']
        df[f'Norway_cos{k}'] = df[f'cos{k}'] * df['country_Norway']
        df[f'store_sin{k}'] = df[f'sin{k}'] * df['store_KaggleMart']
        df[f'store_cos{k}'] = df[f'cos{k}'] * df['store_KaggleMart']
        df[f'mug_sin{k}'] = df[f'sin{k}'] * df['product_Kaggle Mug']
        df[f'mug_cos{k}'] = df[f'cos{k}'] * df['product_Kaggle Mug']
        df[f'sticker_sin{k}'] = df[f'sin{k}'] * df['product_Kaggle Sticker']
        df[f'sticker_cos{k}'] = df[f'cos{k}'] * df['product_Kaggle Sticker']


    ## Easter 
    #2015: 5 April: 95
    #2016: 27 March: 88
    #2017: 16 April: 106
    #2018: 1 April: 91
    #2019: 21 April: 111 
    ## I still look for an easier way to do this, instead of hardcoding it 
    df.loc[df.month>-1,"easter"]=0 ## setting default 
    df.loc[ ( ( (df.year==2018) & ( abs(df.day_of_year-91)<10) ) | 
              ( (df.year==2017) & ( abs(df.day_of_year-106)<10)) | 
              ( (df.year==2016) & ( abs(df.day_of_year-88)<10) ) |
              ( (df.year==2018) & ( abs(df.day_of_year-95)<10) ) |
              ( (df.year==2019) & ( abs(df.day_of_year-111)<10) )
            ),"easter"]=1

    ## Year End 
    df.loc[df.month>0,"year_end"]=0 ## setting default
    df.loc[ ( (  (df.month==12) & (df.day_of_month>22) ) |
                 (  (df.month==1) &  (df.day_of_month <8) ) ),"year_end"]=1

    ## Summer 
    df.loc[df.day_of_year>-1,"summer"]=0 ## ## setting default
    df.loc[ ( (df.day_of_year>125) & (df.day_of_year<190) ),"summer"] =1



    ## we don't need date anymore 
    #df.drop(["date"],axis=1,inplace=True)
    return df 


In [ ]:
df_gdp = pd.read_csv("../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv")


In [ ]:
df_trn = gettimeFeatures(df_trn)
df_tst = gettimeFeatures(df_tst)


## Add GDP 

In [ ]:
df_tst = df_tst.set_index("year").merge( df_gdp.set_index("year"), on= ["year"])
df_trn = df_trn.set_index("year").merge( df_gdp.set_index("year"), on= ["year"])
df_trn.columns


## Add Easter exact date 

In [ ]:
df_tst=df_tst.merge( easter_df,on=["year"])
df_trn=df_trn.merge( easter_df, on=["year"])


In [ ]:
df_tst.reset_index(inplace=True)
df_trn.reset_index(inplace=True)

In [ ]:
def SMAPE(y_true, y_pred):
    diff = np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200
    return diff.mean()


In [ ]:
df_trn = df_trn.set_index(["date"])

## Linear Regression 

In [ ]:
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq="A", order=12)  # 10 sin/cos pairs for "A"nnual seasonality
def getDataFrame(df, country_="", store_="", product_=""):
    df_try = df[ ((df[country_]==1) & (df[store_]==1) & (df[product_]==1))]
    dp = DeterministicProcess(
        index=pd.DatetimeIndex(df_try.index),
        constant=True,               # dummy feature for bias (y-intercept)
        order=1,                     # trend (order 1 means linear)
        seasonal=True,               # weekly seasonality (indicators)
        #additional_terms=[fourier],  # annual seasonality (fourier)
        drop=True,                   # drop terms to avoid collinearity
    )
    X=dp.in_sample()
    Y = df_try['num_sold']
    


    model = LinearRegression(fit_intercept=False)
    _ = model.fit(X, Y)
    y_pred = pd.Series(model.predict(X), index=Y.index)
    
    Y_tmp_pred = pd.DataFrame(y_pred)
    df_try_trn = df_try # just rename
    #print (df_try.head)
    
    ## same for testing dataset 
    from pandas import date_range
    index = date_range("2019-1-1", freq="D", periods=365)
    X_fore = dp.out_of_sample(steps=365,forecast_index=index)
    y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)
    Y_tmp_out = pd.DataFrame(y_fore)
    
    df_try_tst = df_tst[ ((df_tst[country_]==1) & (df_tst[store_]==1) & (df_tst[product_]==1))]
    df_try_tst_1 = df_try_tst.set_index("date")


    k=['Finland','Norway', 'Sweden']
    for ic in k:
        Y_tmp_out[f'country_{ic}'] = (pd.Series((df_try_tst_1[f'country_{ic}'] ).values, index=Y_tmp_out.index))
        Y_tmp_pred[f'country_{ic}'] = (pd.Series((df_try_trn[f'country_{ic}'] ).values, index=Y_tmp_pred.index))

    k_=['KaggleRama', 'KaggleMart']
    for ist in k_:
        Y_tmp_out[f'store_{ist}'] = (pd.Series((df_try_tst_1[f'store_{ist}'] ).values, index=Y_tmp_out.index))
        Y_tmp_pred[f'store_{ist}'] = (pd.Series((df_try_trn[f'store_{ist}'] ).values, index=Y_tmp_pred.index))


    k__=['Kaggle Sticker','Kaggle Hat','Kaggle Mug']
    for ip in k__:
        Y_tmp_out[f'product_{ip}'] = (pd.Series((df_try_tst_1[f'product_{ip}'] ).values, index=Y_tmp_out.index))
        Y_tmp_pred[f'product_{ip}'] = (pd.Series((df_try_trn[f'product_{ip}'] ).values, index=Y_tmp_pred.index))



    return Y_tmp_out, Y_tmp_pred

Ys=[]
Y_preds = []
for ic in ["country_Finland", "country_Sweden", "country_Norway"]:
    for ist in ["store_KaggleRama","store_KaggleMart"]:
        for ip in ["product_Kaggle Sticker","product_Kaggle Hat", "product_Kaggle Mug"]:
            Ys.append(getDataFrame(df_trn,ic, ist, ip)[0])
            Y_preds.append(getDataFrame(df_trn,ic, ist, ip)[1])


Y_out=pd.concat(Ys)
Y_pred_out=pd.concat(Y_preds)


In [ ]:
Y_pred_out

In [ ]:
Y_out.rename({0:"num_sold"},axis=1, inplace=True)
Y_pred_out.rename({0:"num_sold"},axis=1, inplace=True)

In [ ]:
Y_out = Y_out.reset_index()
Y_pred_out = Y_pred_out.reset_index()



In [ ]:
Y_out = Y_out.rename({"index":"date"}, axis=1)
Y_pred_out = Y_pred_out.rename({"index":"date"}, axis=1)




In [ ]:
df_tst.date = pd.to_datetime(df_tst.date)
df_trn.date = pd.to_datetime(df_tst.date)

In [ ]:

Y_out = Y_out.merge(df_tst,on=['date','country_Finland', 'country_Norway',
       'country_Sweden', 'store_KaggleRama', 'store_KaggleMart',
       'product_Kaggle Sticker', 'product_Kaggle Hat', 'product_Kaggle Mug'])
Y_pred_out = Y_pred_out.merge(df_trn,on=['date','country_Finland', 'country_Norway',
       'country_Sweden', 'store_KaggleRama', 'store_KaggleMart',
       'product_Kaggle Sticker', 'product_Kaggle Hat', 'product_Kaggle Mug'])



In [ ]:
Y_out = Y_out[["row_id","num_sold",]]
Y_pred_out = Y_pred_out[["row_id","num_sold_x",]]

In [ ]:
Y_pred_out.sort_values(by="row_id")

In [ ]:
df_trn = df_trn.merge(Y_pred_out,on="row_id")

In [ ]:
df_tst= df_tst.merge(Y_out, on="row_id")

In [ ]:
df_tst.rename({"num_sold":"num_sold_x"}, axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
vars=[ 'store_KaggleMart', 'store_KaggleRama', 'country_Finland',
       'country_Norway', 'country_Sweden', 'product_Kaggle Hat',
       'product_Kaggle Mug', 'product_Kaggle Sticker', 'day_of_year',
       'day_of_month', 'week', 'quarter', 'month', 'year', 'weekd', 'weekend',
       'easter', 'year_end', 'summer','GDP_Finland', 'GDP_Norway', 'GDP_Sweden'#,'num_sold_x'
      #'Finland_sin1', 'Finland_cos1',  'Norway_sin1',  'Norway_cos1', 'store_sin1',
     #'store_cos1',  'mug_sin1',  'mug_cos1',  'sticker_sin1',  'sticker_cos1', 'Finland_sin5',
     #'Finland_cos5','Norway_sin5','Norway_cos5','store_sin5','store_cos5','mug_sin5',
     #'mug_cos5','sticker_sin5','sticker_cos5','Finland_sin13','Finland_cos13','Norway_sin13',
     #'Norway_cos13','store_sin13','store_cos13','mug_sin13','mug_cos13','sticker_sin13',
     #'sticker_cos13','Finland_sin17','Finland_cos17','Norway_sin17','Norway_cos17',
     #'store_sin17','store_cos17','mug_sin17','mug_cos17','sticker_sin17','sticker_cos17',
     #'Finland_sin21','Finland_cos21','Norway_sin21','Norway_cos21','store_sin21',
     #'store_cos21', 'mug_sin21','mug_cos21','sticker_sin21','sticker_cos21','Finland_sin25',
     #'Finland_cos25','Norway_sin25','Norway_cos25','store_sin25','store_cos25',
     #'mug_sin25','mug_cos25','sticker_sin25','sticker_cos25','Finland_sin29',
     #'Finland_cos29','Norway_sin29','Norway_cos29','store_sin29', 'store_cos29',
     #'mug_sin29','mug_cos29','sticker_sin29','sticker_cos29'
]
X = df_trn[df_trn.year<2018][vars]
Y = df_trn[df_trn.year<2018]['num_sold']
clf.fit(X,Y)



In [ ]:
X_val = df_trn[df_trn.year>2017][vars]
Y_val = df_trn[df_trn.year>2017]['num_sold']



In [ ]:

Y_val_predict = clf.predict(X_val) 

X_tst = df_tst[vars]
Y_tst=clf.predict(X_tst)

In [ ]:
Y_val_predict[:10]

In [ ]:
SMAPE(Y_val,Y_val_predict)

In [ ]:
df_gdp_prod =pd.DataFrame(df_gdp.loc[3]/df_gdp.loc[2]).transpose()


## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtree = DecisionTreeRegressor(max_depth=25, min_samples_leaf=0.0001, random_state=3)
dtree.fit(X, Y)
Y_val_predict = dtree.predict(X_val)
print (SMAPE(Y_val,Y_val_predict))

Y_tst_dtree = dtree.predict(X_tst)

In [ ]:
Y_tst_dtree

## Deep NN using Keras 

In [ ]:
X.columns


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=8, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(X, y, epochs=150, batch_size=10)
model.summary()

In [ ]:
X.shape, Y.shape

In [ ]:
#model.fit(X, Y, epochs=150, batch_size=10)



In [ ]:
df_tst["num_sold"] = Y_tst
df_out = df_tst.drop(['store_KaggleMart', 'store_KaggleRama', 'country_Finland',
       'country_Norway', 'country_Sweden', 'product_Kaggle Hat',
       'product_Kaggle Mug', 'product_Kaggle Sticker', 'day_of_year',
       'day_of_month', 'week', 'quarter', 'month', 'year', 'weekd', 'weekend',
       'easter', 'year_end', 'summer','GDP_Finland', 'GDP_Norway', 'GDP_Sweden','easter_day','date',
       'Finland_sin1', 'Finland_cos1',  'Norway_sin1',  'Norway_cos1', 'store_sin1',
     'store_cos1',  'mug_sin1',  'mug_cos1',  'sticker_sin1',  'sticker_cos1', 'Finland_sin5',
     'Finland_cos5','Norway_sin5','Norway_cos5','store_sin5','store_cos5','mug_sin5',
     'mug_cos5','sticker_sin5','sticker_cos5','Finland_sin13','Finland_cos13','Norway_sin13',
     'Norway_cos13','store_sin13','store_cos13','mug_sin13','mug_cos13','sticker_sin13',
     'sticker_cos13','Finland_sin17','Finland_cos17','Norway_sin17','Norway_cos17',
     'store_sin17','store_cos17','mug_sin17','mug_cos17','sticker_sin17','sticker_cos17',
     'Finland_sin21','Finland_cos21','Norway_sin21','Norway_cos21','store_sin21',
     'store_cos21', 'mug_sin21','mug_cos21','sticker_sin21','sticker_cos21','Finland_sin25',
     'Finland_cos25','Norway_sin25','Norway_cos25','store_sin25','store_cos25',
     'mug_sin25','mug_cos25','sticker_sin25','sticker_cos25','Finland_sin29',
     'Finland_cos29','Norway_sin29','Norway_cos29','store_sin29', 'store_cos29',
     'mug_sin29','mug_cos29','sticker_sin29','sticker_cos29','sin1', 'cos1', 'sin5', 'cos5', 'sin9', 'cos9',
       'Finland_sin9', 'Finland_cos9', 'Norway_sin9', 'Norway_cos9',
       'store_sin9', 'store_cos9', 'mug_sin9', 'mug_cos9', 'sticker_sin9',
       'sticker_cos9', 'sin13', 'cos13', 'sin17', 'cos17', 'sin21', 'cos21',
       'sin25', 'cos25', 'sin29', 'cos29','num_sold_x'
                     ],axis=1)


In [ ]:
df_out.columns

In [ ]:
Y_out.sort_values(by="row_id",inplace=True)

In [ ]:
df_out.to_csv("submission.csv",index=False)
#df_out.shape, df_out.columns


In [ ]:
!tail submission.csv 